In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('./submissions.csv')

In [5]:
data.head()

,id,epoch_second,problem_id,contest_id,user_id,language,point,length,result,execution_time
0,11777888,1586677338,abc151_c,abc151,i_lohas_MATCH,C++14 (GCC 5.4.1),300.0,777,AC,73.0
1,10100114,1581749288,abc141_b,abc141,rniya,C++14 (GCC 5.4.1),200.0,1002,AC,1.0
2,1434499,1500261926,abc067_b,abc067,naoki2016,C++14 (GCC 5.4.1),200.0,598,AC,1.0
3,7280619,1567343186,abc139_e,abc139,syym88,Python3 (3.4.3),0.0,1070,TLE,2106.0
4,11168255,1585009689,dwango2017qual_a,dwacon2017-prelims,t_kato,PyPy3 (2.4.0),100.0,99,AC,172.0


In [6]:
users = data['user_id']
unique_users = users.unique()

In [7]:
data_ac = data[data['result'] == 'AC']

In [8]:
data_ac.head()

,id,epoch_second,problem_id,contest_id,user_id,language,point,length,result,execution_time
0,11777888,1586677338,abc151_c,abc151,i_lohas_MATCH,C++14 (GCC 5.4.1),300.0,777,AC,73.0
1,10100114,1581749288,abc141_b,abc141,rniya,C++14 (GCC 5.4.1),200.0,1002,AC,1.0
2,1434499,1500261926,abc067_b,abc067,naoki2016,C++14 (GCC 5.4.1),200.0,598,AC,1.0
4,11168255,1585009689,dwango2017qual_a,dwacon2017-prelims,t_kato,PyPy3 (2.4.0),100.0,99,AC,172.0
5,11197219,1585128140,agc006_b,agc006,zoooma13,C++14 (GCC 5.4.1),400.0,651,AC,38.0


In [9]:
list_for_exec = ['problem_id', 'language', 'execution_time']
list_for_leng =  ['problem_id', 'language', 'length']
data_exec = data_ac[list_for_exec]
data_leng = data_ac[list_for_leng]

In [10]:
group_prob_lang_exec = data_exec.groupby(['language', 'problem_id'])
group_prob_lang_leng = data_leng.groupby(['language', 'problem_id'])

In [11]:
exec_statistics = group_prob_lang_exec.quantile([0.1, 0.25, 0.5, 0.75], interpolation='higher')

In [12]:
exec_statistics.head(20)

execution_time
language             problem_id                     
Ada2012 (GNAT 9.2.1) abc162_a   0.10             0.0
                                0.25             0.0
                                0.50             0.0
                                0.75             0.0
                     abc164_a   0.10             2.0
                                0.25             2.0
                                0.50             2.0
                                0.75             2.0
                     abc164_b   0.10             5.0
                                0.25             5.0
                                0.50             5.0
                                0.75             5.0
                     abc165_a   0.10             3.0
                                0.25             3.0
                                0.50             3.0
                                0.75             3.0
Awk (GNU Awk 4.1.4)  abc162_a   0.10             3.0
                                0.25             4.0
                                0.50             6.0
                                0.75             8.0

In [13]:
length_statistics =  group_prob_lang_leng.quantile([0.1, 0.25, 0.5, 0.75], interpolation='higher')

In [14]:
length_statistics.head(20)

length
language             problem_id             
Ada2012 (GNAT 9.2.1) abc162_a   0.10     187
                                0.25     187
                                0.50     187
                                0.75     187
                     abc164_a   0.10     280
                                0.25     280
                                0.50     280
                                0.75     280
                     abc164_b   0.10     299
                                0.25     299
                                0.50     299
                                0.75     299
                     abc165_a   0.10     483
                                0.25     483
                                0.50     483
                                0.75     483
Awk (GNU Awk 4.1.4)  abc162_a   0.10      17
                                0.25      17
                                0.50      19
                                0.75      20